# **Ingestion del connectivity.json**

##### **01.- Leer archivo JSON usando dataframereader de spark**

In [1]:
#p_file_date = "2025-07-07"
#p_environment = "Development"

StatementMeta(, e0001642-8fec-4357-ab19-4aa2da6b7556, 3, Finished, Available, Finished)

In [2]:
%run configuration

StatementMeta(, e0001642-8fec-4357-ab19-4aa2da6b7556, 4, Finished, Available, Finished)

In [3]:
%run common_functions

StatementMeta(, e0001642-8fec-4357-ab19-4aa2da6b7556, 5, Finished, Available, Finished)

In [4]:
#Importaciones de los tipos de datos para definir el esquema
from pyspark.sql.types import StructType, StructField, IntegerType, DateType, StringType

StatementMeta(, e0001642-8fec-4357-ab19-4aa2da6b7556, 6, Finished, Available, Finished)

In [5]:
# Esta es la estructura del df, esto se define para que inferSchema no haga un proceso extra 
# sirve para grandes volumenes de datos, asi se demoran menos en cargar
display_schema = StructType ( fields = [
    StructField("DisplayID", IntegerType(), False),
    StructField("DisplayHole", StringType(), True),
    StructField("DisplayDiagonal", StringType(), True),
    StructField("Resolution", StringType(), True),
    StructField("DisplayType", StringType(), True),
    StructField("DisplayColorDepth", StringType(), True),
    StructField("DisplayIllumination", StringType(), True),
    StructField("DisplayRefreshRate", StringType(), True)
])

StatementMeta(, e0001642-8fec-4357-ab19-4aa2da6b7556, 7, Finished, Available, Finished)

In [6]:
display_path = f"{bronze_folder_path}/{p_file_date}/display"

df = (
    spark.read
        .schema(display_schema)     
        .option("header", "true") 
        .option("delimiter", ";")
        .csv(display_path)          
)


StatementMeta(, e0001642-8fec-4357-ab19-4aa2da6b7556, 8, Finished, Available, Finished)

##### **2.- Eliminar columnas que no sirven**

In [7]:
df2 = df.drop("DisplayHole", "DisplayDiagonal", "DisplayType")

StatementMeta(, e0001642-8fec-4357-ab19-4aa2da6b7556, 9, Finished, Available, Finished)

##### **3.- Renombrar columnas**

In [8]:
new_cols = ["display_id", "resolution", "color_depth", "illumination", "refresh_rate"]

df3 = df2.toDF(*new_cols)

StatementMeta(, e0001642-8fec-4357-ab19-4aa2da6b7556, 10, Finished, Available, Finished)

##### **4.- Incluir ingestion_date y environment**

In [9]:
from pyspark.sql.functions import lit

df4 = add_ingestion_date(df3) \
        .withColumn("environment", lit(p_environment)) \
        .withColumn("file_date", lit(p_file_date))

StatementMeta(, e0001642-8fec-4357-ab19-4aa2da6b7556, 11, Finished, Available, Finished)

In [10]:
%%sql
DROP TABLE if EXISTS display

StatementMeta(, e0001642-8fec-4357-ab19-4aa2da6b7556, 12, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

##### **5.- Pasar a delta y enviar a lh_silver**

In [11]:
df4.write.format("delta").mode("overwrite").saveAsTable("display")

StatementMeta(, e0001642-8fec-4357-ab19-4aa2da6b7556, 13, Finished, Available, Finished)

In [12]:
%%sql
SELECT * from display
limit(10)

StatementMeta(, e0001642-8fec-4357-ab19-4aa2da6b7556, 14, Finished, Available, Finished)

<Spark SQL result set with 10 rows and 8 fields>